In [13]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm
import dateutil
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics.pairwise import cosine_similarity
import json
import pickle

from keras.models import model_from_json

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error


In [2]:
def mape(y_true,y_predict):
    return np.sum((np.abs(y_true-y_predict)/y_true)*100)/len(y_true)

In [3]:
def load_model(model_name):
    # load json and create model
    json_file = open(model_name+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(model_name+".h5")
    return loaded_model

In [4]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")

In [5]:
cutoff_start_year = '2013'
cutoff_start_date = datetime.strptime(cutoff_start_year+'-01-01','%Y-%m-%d')

In [6]:
match_summary_df = pd.read_csv('csv_data/match_list.csv',parse_dates=['date'],date_parser=custom_date_parser)
recent_match_summary_df = match_summary_df[match_summary_df['date']>=cutoff_start_date]
recent_match_summary_df.shape

(781, 11)

In [7]:
match_stats_df = pd.read_csv('csv_data/match_stats.csv')

In [8]:
recent_match_summary_df=recent_match_summary_df.merge(match_stats_df,on='match_id',how='inner')
recent_match_summary_df=recent_match_summary_df[recent_match_summary_df['first_innings']==recent_match_summary_df['team_statistics']]


In [9]:
recent_match_summary_df.columns

Index(['match_id', 'date', 'location', 'first_innings', 'second_innings',
       'winner', 'win_by', 'win_dif', 'toss_winner', 'player_of_match',
       'train_data', 'team_statistics', 'batsman_1', 'batsman_1_runs',
       'batsman_2', 'batsman_2_runs', 'batsman_3', 'batsman_3_runs',
       'batsman_4', 'batsman_4_runs', 'batsman_5', 'batsman_5_runs',
       'batsman_6', 'batsman_6_runs', 'batsman_7', 'batsman_7_runs',
       'batsman_8', 'batsman_8_runs', 'batsman_9', 'batsman_9_runs',
       'batsman_10', 'batsman_10_runs', 'batsman_11', 'batsman_11_runs',
       'bowler_1', 'bowler_1_wickets', 'bowler_2', 'bowler_2_wickets',
       'bowler_3', 'bowler_3_wickets', 'bowler_4', 'bowler_4_wickets',
       'bowler_5', 'bowler_5_wickets', 'bowler_6', 'bowler_6_wickets',
       'bowler_7', 'bowler_7_wickets', 'bowler_8', 'bowler_8_wickets',
       'bowler_9', 'bowler_9_wickets', 'bowler_10', 'bowler_10_wickets',
       'bowler_11', 'bowler_11_wickets', 'total_run', 'total_wickets'],
     

In [116]:
# for batsman encoding
country_enc_map=pickle.load(open('country_enc_map.pkl','rb'))
batsman_enc_map = pickle.load(open('batsman_enc_map.pkl','rb'))
loc_enc_map_for_batsman = pickle.load(open('loc_enc_map_for_batsman.pkl','rb'))

# for country encoding
loc_enc_map=pickle.load(open('loc_enc_map.pkl','rb'))
group_encode_model_V2 = load_model('group_encode_model_V2')

In [11]:
def get_oh_pos(pos):
    vec=np.zeros((11)).astype(int) 
    vec[pos-1]=1
    return vec

In [12]:
batsman_group_encode_model = load_model('batsman_group_encode_model')

In [14]:
recent_match_summary_df.head()

,match_id,date,location,first_innings,second_innings,winner,win_by,win_dif,toss_winner,player_of_match,...,bowler_8,bowler_8_wickets,bowler_9,bowler_9_wickets,bowler_10,bowler_10_wickets,bowler_11,bowler_11_wickets,total_run,total_wickets
0,589309,2013-01-03,Kolkata,Pakistan,India,Pakistan,runs,85,India,Nasir Jamshed,...,not_bowled,0,not_bowled,0,not_bowled,0,not_bowled,0,250,10
2,589310,2013-01-06,Delhi,India,Pakistan,India,runs,10,India,MS Dhoni,...,not_bowled,0,not_bowled,0,not_bowled,0,not_bowled,0,167,10
4,565812,2013-01-11,Rajkot,England,India,England,runs,9,England,JC Tredwell,...,not_bowled,0,not_bowled,0,not_bowled,0,not_bowled,0,325,9
6,573014,2013-01-11,Melbourne Cricket Ground,Australia,Sri Lanka,Australia,runs,107,Australia,PJ Hughes,...,not_bowled,0,not_bowled,0,not_bowled,0,not_bowled,0,305,10
8,573015,2013-01-13,Adelaide Oval,Australia,Sri Lanka,Sri Lanka,wickets,8,Sri Lanka,HDRL Thirimanne,...,not_bowled,0,not_bowled,0,not_bowled,0,not_bowled,0,170,2


In [14]:
#recent_match_summary_df.iloc[recent_match_summary_df.shape[0]-1]

In [180]:
encoding_train_list = []
encoding_test_list = []

team_encoding_train_list = []
team_encoding_test_list =[]

target_train_list = []
target_test_list =[]

target_sequence_train_list =[]
target_sequence_test_list = []

no_of_rows = recent_match_summary_df.shape[0]
#print(no_of_rows)
for pos in tqdm(range(no_of_rows)):
    match_details = recent_match_summary_df.iloc[pos]
    location = match_details['location']
    team = match_details['first_innings']
    opposition = match_details['second_innings']
    total_run = match_details['total_run']
    is_train = match_details['train_data']
    
    loc_oh = loc_enc_map_for_batsman[location]
    opposition_oh = country_enc_map[opposition]
    
    batsman_oh_list =[]
    position_oh_list =[]
    loc_oh_list =[]
    opposition_oh_list =[]
    
    sequence_output =[]
    #print('getting batsman details')
    for bi in range(11):
        batsman = match_details['batsman_'+str(bi+1)]
        if batsman == 'not_batted':
            #break
            batsman_oh = batsman_enc_map[batsman]
        else:
            batsman_oh = batsman_enc_map[team.strip()+' '+batsman.strip()]
        position_oh = get_oh_pos(bi+1)
        
        batsman_oh_list.append(batsman_oh)
        position_oh_list.append(position_oh)
        loc_oh_list.append(loc_oh)
        opposition_oh_list.append(opposition_oh)
        sequence_output.append(match_details['batsman_'+str(bi+1)+'_runs'])
        
    batsman_mat = np.stack(batsman_oh_list)
    position_mat = np.stack(position_oh_list)
    loc_mat = np.stack(loc_oh_list)
    opposition_mat = np.stack(opposition_oh_list)
    sequence_output_vec = np.array(sequence_output)
    #print('encoding')
    batsman_group_enc_mat = batsman_group_encode_model.predict([batsman_mat,position_mat,loc_mat,opposition_mat])
    #batsman_mean =  batsman_group_enc_mat.sum(axis=0) 
    #batsman_mean = batsman_group_enc_mat.reshape(-1)
    
    ##create team_encoding with V2
    
    team_oh_v = np.array(country_enc_map[team]).reshape(1,-1)
    opponent_oh_v = np.array(opposition_oh).reshape(1,-1)
    if location not in loc_enc_map:
        continue
    loc_oh_v=np.array(loc_enc_map[location]).reshape(1,-1)
    country_enc_vec = group_encode_model_V2.predict([team_oh_v,opponent_oh_v,loc_oh_v]).reshape(-1)
    
    if is_train:
        encoding_train_list.append(batsman_group_enc_mat)
        team_encoding_train_list.append(country_enc_vec)
        target_train_list.append(total_run)
        target_sequence_train_list.append(sequence_output_vec)
    else:
        encoding_test_list.append(batsman_group_enc_mat)
        team_encoding_test_list.append(country_enc_vec)
        target_test_list.append(total_run)
        target_sequence_test_list.append(sequence_output_vec)
    
    #break
    

In [168]:
#np.stack(encoding_train_list).shape

In [181]:
scaler = StandardScaler()

encoding_mat_train = np.stack(encoding_train_list)
team_encoding_mat_train = np.stack(team_encoding_train_list)
#train_x = scaler.fit_transform(encoding_mat_train)
train_y = np.stack(target_train_list)
train_y_seq = np.stack(target_sequence_train_list)

encoding_mat_test = np.stack(encoding_test_list)
team_encoding_mat_test = np.stack(team_encoding_test_list)
#test_x = scaler.transform(encoding_mat_test)
test_y = np.stack(target_test_list)
test_y_seq = np.stack(target_sequence_test_list)

In [182]:
#encoding_mat_train.shape,train_y_seq.shape
#team_encoding_mat_train.shape

((612, 11, 10), (612, 11))

In [171]:
import keras as k
import keras.backend as K
from keras.layers import *
from keras.models import Model
from keras.regularizers import l2

from keras.optimizers import Adam, Adadelta

In [172]:
def create_sequential_model_with_inital_state(timesteps,embedding_lenght,inital_state_vector):
    sequence_input = Input((timesteps,embedding_lenght),name="sequence_input")
    initial_state = Input((inital_state_vector,),name="state_input")
    
    lstm_out = LSTM(inital_state_vector,activation='relu',return_sequences=False,
                    return_state=False, name='lstm_1')(sequence_input,initial_state=[initial_state,initial_state])
    runs_output = Dense(1,name='final_output')(lstm_out)
    
    runs_model = Model(inputs=[sequence_input,initial_state],
                      outputs=runs_output)
    
    return runs_model
    

In [173]:
def create_sequential_model(timesteps,embedding_lenght):
    sequence_input = Input((timesteps,embedding_lenght),name="sequence_input")
    
    
    lstm_out = LSTM(100,activation='relu',return_sequences=False,
                    return_state=False, name='lstm_1')(sequence_input)
#     lstm_out = LSTM(40,activation='relu',return_sequences=False,
#                     return_state=False,name='lstm_2')(lstm_out)
#    lstm_out = Flatten()(lstm_out)
    
    runs_output = Dense(10,name='dense_1',activation='relu')(lstm_out)
#    runs_output = Dense(5,name='dense_2',activation='relu')(runs_output)
    runs_output = Dense(1,name='final_output')(runs_output)
    
    runs_model = Model(inputs=[sequence_input],
                      outputs=runs_output)
    
    return runs_model
    

In [174]:
# runs_model = create_sequential_model(encoding_mat_train.shape[1],encoding_mat_train.shape[2])
runs_model = create_sequential_model_with_inital_state(encoding_mat_train.shape[1],
                                                       encoding_mat_train.shape[2],
                                                      team_encoding_mat_train.shape[1])

In [175]:
runs_model.summary()

Model: "functional_43"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence_input (InputLayer)     [(None, 11, 10)]     0                                            
__________________________________________________________________________________________________
state_input (InputLayer)        [(None, 30)]         0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 30)           4920        sequence_input[0][0]             
                                                                 state_input[0][0]                
                                                                 state_input[0][0]                
______________________________________________________________________________________

In [176]:
runs_model.compile(loss="mean_squared_error", metrics=["mean_absolute_error","mean_absolute_percentage_error"],optimizer=Adam(0.00001))

In [177]:
# runs_model.fit([encoding_mat_train], train_y,
#                validation_data=([encoding_mat_test],test_y),epochs=100, batch_size=10)

runs_model.fit([encoding_mat_train,team_encoding_mat_train], train_y,
               validation_data=([encoding_mat_test,team_encoding_mat_test],test_y),epochs=100, batch_size=10)

Epoch 1/100
62/62 [==============================] - 1s 12ms/step - loss: 70890.3438 - mean_absolute_error: 257.8497 - mean_absolute_percentage_error: 100.0289 - val_loss: 73423.7734 - val_mean_absolute_error: 262.2694 - val_mean_absolute_percentage_error: 99.9971
Epoch 2/100
62/62 [==============================] - 0s 8ms/step - loss: 70792.3828 - mean_absolute_error: 257.6637 - mean_absolute_percentage_error: 99.9523 - val_loss: 73322.8281 - val_mean_absolute_error: 262.0791 - val_mean_absolute_percentage_error: 99.9171
Epoch 3/100
62/62 [==============================] - 0s 8ms/step - loss: 70684.7031 - mean_absolute_error: 257.4582 - mean_absolute_percentage_error: 99.8674 - val_loss: 73209.5547 - val_mean_absolute_error: 261.8653 - val_mean_absolute_percentage_error: 99.8272
Epoch 4/100
62/62 [==============================] - 0s 7ms/step - loss: 70559.0859 - mean_absolute_error: 257.2193 - mean_absolute_percentage_error: 99.7686 - val_loss: 73073.0312 - val_mean_absolute_error: 2

62/62 [==============================] - 1s 9ms/step - loss: 3030.1628 - mean_absolute_error: 42.8467 - mean_absolute_percentage_error: 20.2927 - val_loss: 4391.2148 - val_mean_absolute_error: 50.9096 - val_mean_absolute_percentage_error: 24.5477
Epoch 64/100
62/62 [==============================] - 1s 10ms/step - loss: 3019.4148 - mean_absolute_error: 42.9019 - mean_absolute_percentage_error: 20.0388 - val_loss: 4355.4463 - val_mean_absolute_error: 50.5614 - val_mean_absolute_percentage_error: 24.5330
Epoch 65/100
62/62 [==============================] - 1s 11ms/step - loss: 3013.4338 - mean_absolute_error: 42.8448 - mean_absolute_percentage_error: 20.0767 - val_loss: 4344.9517 - val_mean_absolute_error: 50.4460 - val_mean_absolute_percentage_error: 24.5242
Epoch 66/100
62/62 [==============================] - 0s 7ms/step - loss: 3012.9568 - mean_absolute_error: 42.8043 - mean_absolute_percentage_error: 20.1130 - val_loss: 4328.3857 - val_mean_absolute_error: 50.2733 - val_mean_absolu

In [179]:
#team_encoding_mat_train.shape

# Seq-2-Seq

In [226]:
def create_seq2seq_model_with_inital_state(timesteps,embedding_lenght,inital_state_vector):
    sequence_input = Input((timesteps,embedding_lenght),name="sequence_input")
    initial_state = Input((inital_state_vector,),name="state_input")
    
    lstm_out,state_h,state_c = LSTM(inital_state_vector,activation='relu',return_sequences=True,
                    return_state=True, name='lstm_1')(sequence_input,initial_state=[initial_state,initial_state])
    runs_output = TimeDistributed(Dense(1,name='ts_individual_output'))(lstm_out)
    
    runs_output = Flatten(name='individual_output')(runs_output)
    total_runs_output = Dense(1,name='total_output')(state_h)

    
    runs_model = Model(inputs=[sequence_input,initial_state],
                      outputs=[runs_output,total_runs_output])
    
    return runs_model

In [227]:
runs_model = create_seq2seq_model_with_inital_state(encoding_mat_train.shape[1],
                                                       encoding_mat_train.shape[2],
                                                      team_encoding_mat_train.shape[1])

In [228]:
runs_model.summary()

Model: "functional_64"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence_input (InputLayer)     [(None, 11, 10)]     0                                            
__________________________________________________________________________________________________
state_input (InputLayer)        [(None, 30)]         0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 11, 30), (No 4920        sequence_input[0][0]             
                                                                 state_input[0][0]                
                                                                 state_input[0][0]                
______________________________________________________________________________________

In [229]:
runs_model.compile(loss="mean_squared_error", metrics=["mean_absolute_error"],optimizer=Adam(0.00001))

In [230]:
runs_model.fit([encoding_mat_train,team_encoding_mat_train], [train_y_seq,train_y],
               validation_data=([encoding_mat_test,team_encoding_mat_test],[test_y_seq,test_y]),epochs=100, batch_size=10)

Epoch 1/100
62/62 [==============================] - 1s 13ms/step - loss: 72976.8516 - individual_output_loss: 1239.4268 - total_output_loss: 71737.4219 - individual_output_mean_absolute_error: 20.2514 - total_output_mean_absolute_error: 259.4413 - val_loss: 75352.4141 - val_individual_output_loss: 1216.5403 - val_total_output_loss: 74135.8672 - val_individual_output_mean_absolute_error: 20.6672 - val_total_output_mean_absolute_error: 263.6002
Epoch 2/100
62/62 [==============================] - 0s 7ms/step - loss: 72632.7109 - individual_output_loss: 1243.9147 - total_output_loss: 71388.7891 - individual_output_mean_absolute_error: 20.2861 - total_output_mean_absolute_error: 258.7934 - val_loss: 75025.4766 - val_individual_output_loss: 1218.6393 - val_total_output_loss: 73806.8438 - val_individual_output_mean_absolute_error: 20.6798 - val_total_output_mean_absolute_error: 262.9850
Epoch 3/100
62/62 [==============================] - 1s 8ms/step - loss: 72331.4062 - individual_output_l

62/62 [==============================] - 1s 9ms/step - loss: 5085.8413 - individual_output_loss: 1862.3895 - total_output_loss: 3223.4517 - individual_output_mean_absolute_error: 34.2906 - total_output_mean_absolute_error: 44.8593 - val_loss: 6524.1646 - val_individual_output_loss: 1664.7231 - val_total_output_loss: 4859.4414 - val_individual_output_mean_absolute_error: 31.9203 - val_total_output_mean_absolute_error: 55.2006
Epoch 38/100
62/62 [==============================] - 0s 8ms/step - loss: 4948.9424 - individual_output_loss: 1696.8263 - total_output_loss: 3252.1140 - individual_output_mean_absolute_error: 32.3956 - total_output_mean_absolute_error: 45.0630 - val_loss: 6409.8169 - val_individual_output_loss: 1520.4659 - val_total_output_loss: 4889.3496 - val_individual_output_mean_absolute_error: 30.2309 - val_total_output_mean_absolute_error: 55.3562
Epoch 39/100
62/62 [==============================] - 0s 7ms/step - loss: 4838.6758 - individual_output_loss: 1538.5311 - total_o

Epoch 56/100
62/62 [==============================] - 1s 12ms/step - loss: 4000.1831 - individual_output_loss: 1033.5197 - total_output_loss: 2966.6636 - individual_output_mean_absolute_error: 21.8705 - total_output_mean_absolute_error: 42.9152 - val_loss: 5407.2666 - val_individual_output_loss: 1021.2455 - val_total_output_loss: 4386.0210 - val_individual_output_mean_absolute_error: 22.2547 - val_total_output_mean_absolute_error: 51.3136
Epoch 57/100
62/62 [==============================] - 1s 8ms/step - loss: 4001.3484 - individual_output_loss: 1038.5573 - total_output_loss: 2962.7920 - individual_output_mean_absolute_error: 21.9673 - total_output_mean_absolute_error: 42.9357 - val_loss: 5410.0571 - val_individual_output_loss: 1008.8979 - val_total_output_loss: 4401.1597 - val_individual_output_mean_absolute_error: 21.9797 - val_total_output_mean_absolute_error: 51.5326
Epoch 58/100
62/62 [==============================] - 1s 14ms/step - loss: 3982.3779 - individual_output_loss: 1031

Epoch 93/100
62/62 [==============================] - 1s 12ms/step - loss: 3904.6033 - individual_output_loss: 986.7886 - total_output_loss: 2917.8149 - individual_output_mean_absolute_error: 20.6323 - total_output_mean_absolute_error: 42.4497 - val_loss: 5424.3535 - val_individual_output_loss: 961.7136 - val_total_output_loss: 4462.6392 - val_individual_output_mean_absolute_error: 20.7731 - val_total_output_mean_absolute_error: 51.3547
Epoch 94/100
62/62 [==============================] - 1s 11ms/step - loss: 3931.7532 - individual_output_loss: 994.9815 - total_output_loss: 2936.7717 - individual_output_mean_absolute_error: 20.7707 - total_output_mean_absolute_error: 42.3712 - val_loss: 5466.5435 - val_individual_output_loss: 956.0642 - val_total_output_loss: 4510.4790 - val_individual_output_mean_absolute_error: 20.6534 - val_total_output_mean_absolute_error: 51.9485
Epoch 95/100
62/62 [==============================] - 1s 10ms/step - loss: 3916.8440 - individual_output_loss: 984.319

In [233]:
#runs_model.predict([encoding_mat_train,team_encoding_mat_train])